# Lab | Handling Data Imbalance in Classification Models


You are working as an analyst with this internet service provider. You are provided with this historical data about your company's customers and their churn trends. Your task is to build a machine learning model that will help the company identify customers that are more likely to default/churn and thus prevent losses from such customers.

In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned on the class.

#### Import the required libraries and modules that you would need. Read that data into Python and call the dataframe churnData.

In [114]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler


In [115]:
churnData = pd.read_csv(r'C:\Student\IRONHACK\Week7\D3_ImbalancedData_DecissionTrees_CrossValidation\Lab\lab-handling-data-imbalance-classification\files_for_lab\Customer-Churn.csv')
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


#### Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.

In [116]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [117]:
churnData['TotalCharges'] = churnData['TotalCharges'].apply(lambda x: 0 if x == ' ' else x)
pd.to_numeric(churnData['TotalCharges'])

0         29.85
1       1889.50
2        108.15
3       1840.75
4        151.65
         ...   
7038    1990.50
7039    7362.90
7040     346.45
7041     306.60
7042    6844.50
Name: TotalCharges, Length: 7043, dtype: float64

#### Check for null values in the dataframe. Replace the null values.

In [118]:
churnData.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

#### Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges. Scale the features either by using normalizer or a standard scaler.

In [119]:
selection = churnData[['tenure', 'SeniorCitizen', 'MonthlyCharges','TotalCharges']]

In [120]:
# To make run all you ask in one cell, not only the last required
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [121]:
# scaling standard scaler: make data normal distributed with mean=0 and std=1
transformer = StandardScaler().fit(selection)
sel_standardized = transformer.transform(selection)
print(sel_standardized.shape)
sel_stand = pd.DataFrame(sel_standardized, columns=selection.columns)
sel_stand.describe()

(7043, 4)


,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
count,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03
mean,5.945991e-17,6.417792e-16,-8.291599e-17,-2.667184e-17
std,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00
min,-1.318165e+00,-4.399165e-01,-1.545860e+00,-1.005780e+00
25%,-9.516817e-01,-4.399165e-01,-9.725399e-01,-8.299464e-01
50%,-1.372744e-01,-4.399165e-01,1.857327e-01,-3.905282e-01
75%,9.214551e-01,-4.399165e-01,8.338335e-01,6.648034e-01
max,1.613701e+00,2.273159e+00,1.794352e+00,2.825806e+00


#### Split the data into a training set and a test set.

In [122]:
X = sel_stand
y = churnData['Churn']

In [123]:
X.shape
y.shape

(7043, 4)

(7043,)

In [124]:
# Splitting into train set and test set.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)
X_train.head()

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
6630,-0.666639,2.273159,0.464915,-0.468309
7013,0.310650,-0.439916,0.951821,0.651480
2224,-0.625919,2.273159,0.388472,-0.461625
6580,0.677133,-0.439916,0.439988,0.681392
1501,-0.788800,-0.439916,-1.489359,-0.898286


#### Fit a logistic regression model on the training data.

In [125]:
from sklearn.linear_model import LogisticRegression

In [126]:
import warnings
warnings.filterwarnings('ignore')
classification = LogisticRegression(random_state=0, solver='saga',
                  multi_class='multinomial').fit(X_train, y_train)

In [127]:
predictions = classification.predict(X_test)
classification.score(X_test, y_test)

0.7762634866553095

#### Check the accuracy on the test data.

In [128]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[1150,  122],
       [ 272,  217]], dtype=int64)

### Managing imbalance in the dataset

#### Check for the imbalance.
Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
Each time fit the model and see how the accuracy of the model is.

In [129]:
churnData['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [130]:
from sklearn.utils import resample

category_0 = churnData[churnData['Churn'] == 'No']
category_1 = churnData[churnData['Churn'] == 'Yes']

In [131]:
category_1_oversampled = resample(category_1, 
                                  replace=True, 
                                  n_samples = len(category_0))

In [132]:
print(category_0.shape)
print(category_1_oversampled.shape)

(5174, 16)
(5174, 16)


In [133]:
data_upsampled = pd.concat([category_0, category_1_oversampled], axis=0)

In [134]:
data_upsampled['Churn'].value_counts()

Yes    5174
No     5174
Name: Churn, dtype: int64

In [135]:
category_0_downsampled = resample(category_0, 
                                  replace=False, 
                                  n_samples = len(category_1))

In [136]:
print(category_1.shape)
print(category_0_downsampled.shape)

(1869, 16)
(1869, 16)


In [137]:
data_downsampled = pd.concat([category_1, category_0_downsampled], axis=0)

In [138]:
data_downsampled['Churn'].value_counts()

Yes    1869
No     1869
Name: Churn, dtype: int64

#### Apply logistic regression model to oversampled data

In [139]:
# data_upsampled.head()

In [140]:
selection = data_upsampled[['tenure', 'SeniorCitizen', 'MonthlyCharges','TotalCharges']]

In [141]:
selection.head()

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
0,1,0,29.85,29.85
1,34,0,56.95,1889.5
3,45,0,42.30,1840.75
6,22,0,89.10,1949.4
7,10,0,29.75,301.9


In [142]:
# scaling standard scaler: make data normal distributed with mean=0 and std=1
transformer = StandardScaler().fit(selection)
sel_standardized = transformer.transform(selection)
print(sel_standardized.shape)
sel_stand = pd.DataFrame(sel_standardized, columns=selection.columns)
sel_stand.describe()

(10348, 4)


,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
count,1.034800e+04,1.034800e+04,1.034800e+04,1.034800e+04
mean,9.041880e-15,-2.062088e-16,7.637665e-16,-1.053360e-16
std,1.000048e+00,1.000048e+00,1.000048e+00,1.000048e+00
min,-1.161362e+00,-4.891640e-01,-1.730633e+00,-9.404641e-01
25%,-9.532957e-01,-4.891640e-01,-7.802449e-01,-8.124903e-01
50%,-2.458710e-01,-4.891640e-01,2.343114e-01,-4.001993e-01
75%,8.776857e-01,-4.891640e-01,7.944852e-01,5.605982e-01
max,1.834790e+00,2.044304e+00,1.755279e+00,3.041537e+00


In [143]:
X = sel_stand
y = data_upsampled['Churn']

In [144]:
# Splitting into train set and test set.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)
X_train.head()

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
4614,-0.037805,2.044304,0.615854,0.077457
6010,0.170261,2.044304,0.595043,0.279657
741,-0.620390,-0.489164,-1.669933,-0.817379
7247,-0.870069,-0.489164,0.376523,-0.700690
3126,-1.119749,-0.489164,-1.671667,-0.931317


In [145]:
import warnings
warnings.filterwarnings('ignore')
classification = LogisticRegression(random_state=0, solver='saga',
                  multi_class='multinomial').fit(X_train, y_train)

In [146]:
predictions = classification.predict(X_test)
classification.score(X_test, y_test)

0.7309625048318515

In [147]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[944, 337],
       [359, 947]], dtype=int64)

#### Apply logistic regression model to downsampled data

In [148]:
selection = data_downsampled[['tenure', 'SeniorCitizen', 'MonthlyCharges','TotalCharges']]

In [149]:
# scaling standard scaler: make data normal distributed with mean=0 and std=1
transformer = StandardScaler().fit(selection)
sel_standardized = transformer.transform(selection)
print(sel_standardized.shape)
sel_stand = pd.DataFrame(sel_standardized, columns=selection.columns)
sel_stand.describe()

(3738, 4)


,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
count,3.738000e+03,3.738000e+03,3.738000e+03,3.738000e+03
mean,-2.978801e-15,-1.416500e-15,-4.440595e-16,-2.376079e-18
std,1.000134e+00,1.000134e+00,1.000134e+00,1.000134e+00
min,-1.155849e+00,-4.896948e-01,-1.718284e+00,-9.396201e-01
25%,-9.466226e-01,-4.896948e-01,-7.807995e-01,-8.200333e-01
50%,-2.770988e-01,-4.896948e-01,2.273568e-01,-4.031120e-01
75%,8.945679e-01,-4.896948e-01,7.992706e-01,5.749373e-01
max,1.857008e+00,2.042088e+00,1.768165e+00,3.054179e+00


In [154]:
X = sel_stand
y = data_downsampled['Churn']

In [155]:
# Splitting into train set and test set.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)
X_train.head()

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
706,-1.072158,-0.489695,0.295411,-0.869813
2032,1.605937,-0.489695,-0.229825,0.988760
3221,1.271175,-0.489695,0.274471,1.091010
3500,-0.946623,2.042088,0.073799,-0.800466
1499,-0.444480,-0.489695,-1.529828,-0.751123


In [156]:
import warnings
warnings.filterwarnings('ignore')
classification = LogisticRegression(random_state=0, solver='saga',
                  multi_class='multinomial').fit(X_train, y_train)

In [157]:
predictions = classification.predict(X_test)
classification.score(X_test, y_test)

0.758288770053476

In [158]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[344, 114],
       [112, 365]], dtype=int64)